# The Neurogenomics Database: Dotplot of entire dataset predictions
Author: Nienke Mekkes <br>
Date: 11-10-2022. <br>
Correspond: n.j.mekkes@umcg.nl <br>

## Script: Dotplot of entire dataset predictions
Build Dot Plots for each diagnosis category.


### Input files:
- predictions file
- metadata about attributes (for correct order and grouping)
- output folder
- information about donors
- expectations about symptomatology



## IMPORTANT

this script works with a clinical trajectory dictionary pickle. this pickle can be a rules of thumb or a original pickle, which was generated by the script proces_predictions. These pickles were not prefiltered on donorlevel, meaning they can be used by both the paper and the 'total website'. This means that in the original predictions, weird sentences etc. and the 10 attributes that perform poorly are removed. However, donors that you want to be excluded have to be excluded. This can be done in two ways: <br>
1. in this script, manually. for example remove donors younger than 21. or donors with the NAD diagnosis, or reassign diagnosis (e.g. a SSA, CON donor NBB xxx needs to become HIV).
2. with an input file, for example an excel file that contains minimally one column with donorids, and one column with adiagnosis. diagnosis can be an updated diagnosis (for example, NBB xxxx and diagnosis HIV), or the word 'excluded' when you want to exclude it

## PATHS

In [ ]:
path_to_predictions = ""
path_to_attribute_grouping = ""
figure_folder = ""
expected_attributes_path = ""
general_information = ""

path_to_cleaned_training_data = ""

train_plot = False

### IMPORTS

In [ ]:

import seaborn as sns; sns.set()
import matplotlib
import numpy as np; np.random.seed(0)
from matplotlib import pyplot as plt 
import xlsxwriter
import pandas as pd
import os
import numpy as np
import scattermap
from scattermap import scattermap
import pickle
import multiprocessing
import statsmodels
from functools import partial
from multiprocessing import Pool
import sys
import scipy
from helper_functions import permutation_of_individual_test, table_selector
import datetime

In [ ]:
print(sns.__version__)
print(matplotlib.__version__)
print(statsmodels.__version__)
print(scipy.__version__)

In [ ]:
if not os.path.exists(figure_folder):
    print('Creating output folder....')
    os.makedirs(figure_folder)

### Load data

In [ ]:
if train_plot == False:
    with open(path_to_predictions,"rb") as file:
        predictions_pickle = pickle.load(file)

    d = []
    for i,j in zip(predictions_pickle,predictions_pickle.values()):
        k = pd.DataFrame.from_dict(j,orient="index")
        k["DonorID"] = i
        k['Age'] = k.index
        d.append(k)

    predictions_df =pd.concat(d, ignore_index=True)
    display(predictions_df)
    print(f"there are {len(list(predictions_df['DonorID'].unique()))} unique donor IDs")
    print(predictions_df.shape)

In [ ]:
if train_plot == True:
    with open(path_to_cleaned_training_data,"rb") as file:
        predictions_pickle = pickle.load(file)
        predictions_df = pd.DataFrame(predictions_pickle)

    predictions_df = predictions_df.rename(columns={"NBB_nr": "DonorID"})
    predictions_df.drop(['Year_Sentence_nr'], axis=1,inplace=True,  errors='ignore')
    display(predictions_df)   

### exclude/change donors for the paper, using general info
- under 21
- NAD diagnosis

In [ ]:
general_information_df = pd.read_excel(general_information, engine='openpyxl', sheet_name="Sheet1")
donors_to_remove = list(general_information_df[general_information_df['paper diagnosis']=='exclude'].DonorID)
predictions_df = predictions_df[~predictions_df['DonorID'].isin(donors_to_remove)]
print(f"there are {len(list(predictions_df['DonorID'].unique()))} unique donor IDs")
print(len(donors_to_remove))
predictions_df['neuropathological_diagnosis'] = predictions_df['DonorID'].map(general_information_df.set_index('DonorID')['paper diagnosis'])
display(predictions_df.head())
print(sorted(predictions_df['neuropathological_diagnosis'].unique()))
print(f"there are {len(list(predictions_df['DonorID'].unique()))} unique donor IDs")


In [ ]:
if train_plot == False:
    non_attribute_columns = ['DonorID','Year','age_at_death','sex',
                            'neuropathological_diagnosis','Age'] #'birthyear',,'death_year','year_before_death','sex',
if train_plot == True:
    non_attribute_columns = ['DonorID','neuropathological_diagnosis','Sentence'] #'birthyear',,'death_year','year_before_death','sex',
attributes = [col for col in predictions_df.columns if col not in non_attribute_columns]
# display(attributes)
print(f"there are {predictions_df.shape[0]} rows and {len(attributes)} attributes")
print(f"there are {len(list(predictions_df['DonorID'].unique()))} unique donor IDs")

In [ ]:
foo = predictions_df[predictions_df['DonorID'] == 'NBB 2020-013']
foo['Dementia']

## setting future dotplot colors based on domain or grouping
The attributes all belong to different 'domains' and subgroups. we color them based on these grouping. <br>

We make two dictionaries with the attributes (values) grouped with their domains (keys). One with the finalised names, this dictionary will be used to update the attribute names. The other one has the 'pc_friendly' names, these are used

In [ ]:
attribute_grouping = pd.read_excel(path_to_attribute_grouping, engine='openpyxl', index_col=[0], sheet_name='90 parameters')
attribute_grouping

### based on grouping

In [ ]:
group_dict_fancy = dict()
color_dict_fancy = dict()
count = 0

colors = {'Aspecific_symptoms':'#ce6dbd',#'Aspecific_symptoms'
          'Autonomic_dysfunction':'#b5cf6b',#'Autonomic_dysfunction'
         'Cerebral_vestibular_system_dysfunction': '#6b6ecf',#'Cerebral_vestibular_system_dysfunction'
         'Changes_in_consciousness_awareness_orientation': '#d6616b',# Changes_in_consciousness_awareness_orientation
         'cognitive_and_memory_impairment':'#e7ba52',#'cognitive_and_memory_impairment'
          '(Dis)inhibition':'#bd9e39',#'Disinhibition'
          'Disturbances_in_mood_behaviour':'#ad494a',#Disturbances_in_mood_behaviour
          'Extrapyramidal_signs_symptoms':'#9c9ede',#Extrapyramidal_signs_symptoms
          'General_decline':'#a55194',#
          'Impaired_mobility':'#393b79',#Mobility_problems
          'Motor_deficit':'#5254a3',#Motor_deficit
         'oth_signs_symptoms_cortical_dysfunction': '#e7cb94',#oth_signs_symptoms_cortical_dysfunction
        'other_psychiatric_signs_symptoms':  '#e7969c',#other_psychiatric_signs_symptoms
          'Sensory_deficits':'#8ca252'}#Sensory_deficits]


    
for attr, group in zip(attribute_grouping.index, attribute_grouping["Grouping"]):
    if group not in group_dict_fancy:
#         print(f"{attr} belonging to {group}, grouping is new")
        if not isinstance(group, float):
            group_dict_fancy[group] = []
            color_dict_fancy[group] = colors[group]
#             print(colors[group])
#             print(color_dict_fancy)
            group_dict_fancy[group].append(attr)
            count +=1
    else:
        group_dict_fancy[group].append(attr)
        
print(group_dict_fancy, '\n')
print(color_dict_fancy)

#### Define order of displaying groupings/domains

In [ ]:
domain_order = ['Aspecific_symptoms',
                'General_decline',
                'Extrapyramidal_signs_symptoms',
                'Cerebral_vestibular_system_dysfunction',
                'Motor_deficit',
                'Impaired_mobility',
                'Autonomic_dysfunction',
                'Sensory_deficits',
                'oth_signs_symptoms_cortical_dysfunction',
                'cognitive_and_memory_impairment',
                '(Dis)inhibition',
                'other_psychiatric_signs_symptoms',
                'Changes_in_consciousness_awareness_orientation',
                'Disturbances_in_mood_behaviour',
               ]

new_order_fancy = []
for x in domain_order:
    group_fancy = group_dict_fancy[x]
    for attr in group_fancy:
        new_order_fancy.append(attr)
# new_order_fancy

#### These attributes perform poorly and need to be removed!

In [ ]:
if train_plot == False:
    new_order_fancy.remove('Unspecified disturbed gait patterns')
    new_order_fancy.remove('Loss of sympathy / empathy')
    new_order_fancy.remove('Headturning sign')
    new_order_fancy.remove('Impaired comprehension')
    new_order_fancy.remove('Changed behavior/personality')
    new_order_fancy.remove('Frontal release signs')
    # new_order_fancy

    # remove 4 synonyms
    new_order_fancy.remove('Ataxia')
    new_order_fancy.remove('Lack of initiative')
    new_order_fancy.remove('Lack of planning / organization / overview')
    new_order_fancy.remove('Cognitive decline')

#### We want to display the attributes using the official names from google drive

In [ ]:
correct_names = {}

for attr, real_name in zip(attribute_grouping.index, attribute_grouping["Attribute"]):
    if not isinstance(real_name, float):
        correct_names[real_name] = attr
# correct_names

#### rename columns

In [ ]:
predictions_df = predictions_df.rename(correct_names,axis=1)

#### change the attribute order to the one we want to display

In [ ]:
information_from_symptoms_df = predictions_df[non_attribute_columns]
attribute_columns_to_sort = predictions_df.loc[:,[i for i in list(predictions_df.columns) if i not in non_attribute_columns]]
attribute_columns_to_sort = attribute_columns_to_sort[new_order_fancy]
          
              
# updated symptoms_df, now with the right columns order
predictions_df = pd.concat([information_from_symptoms_df, attribute_columns_to_sort], axis=1)
display(predictions_df)

### Selecting groups of diagnoses to display in dotplot

We do not want to print all hundreds of diagnoses, but make a selection. here we use a dictionary approach to select diagnoses and give them an appropriate abbreviation in one go <br>

For the statistical permutation test we have two different approaches: <br>
table1: main diagnosis, the background is table 1 without the current diagnosis <br>
table 2 t/m 7: the background is table 1 where all diagnoses belonging to the table of interest are removed.


#### make the main diagnosis readable

In [ ]:
# symptoms_df['Main_diagnosis'] = symptoms_df['Main_diagnosis'].str.strip('[]').str.strip('"').str.strip("'").str.replace(' ', '')
xs = list(predictions_df.neuropathological_diagnosis.unique())
xs[0:10]

#### select sentences from the predictions file

In [ ]:
table_of_choice = 'table1_p' #fig 4a table3_with_con_p #table2_p #fig 3a table1_P fig sup 5a:table2_p

In [ ]:
selected_diagnoses,ordered_diagnoses = table_selector(table_of_choice, predictions_df)
print('After selecting for {}, we have {} donors'.format(selected_diagnoses['neuropathological_diagnosis'].unique(),
                                                                                    selected_diagnoses['DonorID'].nunique()) )
display(ordered_diagnoses)


In [ ]:
table1, _ = table_selector('table1_p', predictions_df)
print('After selecting for {}, we have {} donors'.format(table1['neuropathological_diagnosis'].unique(),
                                                                                    table1['DonorID'].nunique()) )


In [ ]:
display(selected_diagnoses)


In [ ]:
##DonorID	Year			neuropathological_diagnosis	
if train_plot == False:
    table1.drop(['Year','age_at_death','sex','Age'], axis=1,inplace=True, errors='ignore')
    selected_diagnoses.drop(['Year','age_at_death','sex','Age'], axis=1,inplace=True,  errors='ignore')
if train_plot == True:
    table1.drop(['Sentence'], axis=1,inplace=True, errors='ignore') ## for training data
    selected_diagnoses.drop(['Sentence'], axis=1,inplace=True,  errors='ignore') ## for training data

flattened_t1 = table1.groupby(['DonorID','neuropathological_diagnosis'], as_index=False).sum()
flattened = selected_diagnoses.groupby(['DonorID','neuropathological_diagnosis'], as_index=False).sum()
display(flattened)
selected_donorcountpredict = table1['DonorID'].nunique()
# print(selected_donorcountpredict)

print(flattened['neuropathological_diagnosis'].value_counts())
print(flattened_t1['neuropathological_diagnosis'].value_counts())

In [ ]:
disease_counts = pd.DataFrame(flattened['neuropathological_diagnosis'].value_counts())
disease_counts = disease_counts.reindex(ordered_diagnoses)
display(disease_counts)
print(disease_counts.shape[0])
print(disease_counts.loc[['CON']])

In [ ]:
## endgoal: divide total nr of donors in a diagnosis group by the nr of donors suffering from attribute x
## step 1: make boolean df: a donor either has a attribute or not, i dont care about how often.
## by setting it to 1, we can sum and see how many donors have this attribute
group_ready = flattened.copy()
group_ready.loc[:,[i for i in list(predictions_df.columns) if i not in non_attribute_columns]] = group_ready.loc[:,[i for i in list(predictions_df.columns) if i not in non_attribute_columns]].apply(lambda x: [y if y <= 1 else 1 for y in x])
proportion_df = group_ready.groupby('neuropathological_diagnosis').sum()

## now we want to know the total nr of donors in a diagnosis group:
proportion_df['total'] = disease_counts 
proportion_df=proportion_df.reindex(ordered_diagnoses)
# display(proportion_df)

## divide by this total and multiple for percentage
## ofcourse true percentage is multiplied by 100. we multiply by 300 purely for visualization purposes (== bigger circles)
proportion_df.loc[:,[i for i in list(predictions_df.columns) if i not in non_attribute_columns]] = proportion_df.loc[:,[i for i in list(predictions_df.columns) if i not in non_attribute_columns]].div(proportion_df.total, axis=0) * 300
proportion_df = proportion_df.drop(['total'],axis=1)
display(proportion_df)
proportion_df.to_excel(f"{figure_folder}/{table_of_choice}/percentages.xlsx")
display(proportion_df['Dementia'])


In [ ]:
## calculate mean value
general_mean_multi = flattened.groupby('neuropathological_diagnosis').mean()
general_mean_multi = general_mean_multi.reindex(ordered_diagnoses)
display(general_mean_multi)
print(general_mean_multi.max())

### Permutation testing
Only run once, takes a lot of time if you dont have resources. can be skipped if you dont need significane, then do not plot significance in the visualization or you will get an error. Can also be faster if you run less permutations. standard set to 100.000

In [ ]:
def identify_diagnosis_Pvalues(pd, occurence,mdf,dc,t1o):
    
    ##Function that is a wrapper around permutation_of_individual_test and saves all P-values
    ##Make a diagnosis dictionary with a attrobite dictionary that contains all the P-values 
    p_values_diagnosis_dictionary = {} 
    perms = 100000
#     temp_pd = pd[0:2]
    nr = 1
    for d in pd:
        ##Print messages
        message = '--------------------------------------- \n \
                   Working on {primary_diagnosis}, {v} out of {len_pd}'.format(primary_diagnosis=d,v=nr,len_pd=len(pd))
        print(message)
        nr = nr + 1
#         now = datetime.now()
#         current_time = now.strftime("%H:%M:%S")
#         print("Current Time =", current_time)
        nr_donors_with_d = int(dc.loc[d]['neuropathological_diagnosis'])
        print('diagnosis affects {} donors.'.format(nr_donors_with_d))
        # multiproc
        p_values_attribute_dictionary = {} 
        iterable = [attribute_nr for attribute_nr in range(2,occurence.shape[1])]
#         print(iterable)
        pool = multiprocessing.Pool(multiprocessing.cpu_count()-1)
#         mom = 'mean'
#         if mom == 'mean':
        func = partial(permutation_of_individual_test, d, occurence, mdf, nr_donors_with_d,perms,t1o)
#         elif mom == 'median':
#             func = partial(permutation_of_individual_test, d,  occurence, mdf, nr_donors_with_d,t1o,m_or_m='median')
        res = pool.map(func, iterable)
        pool.close()
        pool.join()
        
        p_values_diagnosis_dictionary[d] = res
        
#         ## without multiproc
#         p_values_attribute_dictionary = {} 

#         for attribute_nr in range(2,occurence.shape[1]): #range(2,6):
#             message2 = 'Working on attribute {nr}: {attribute}'.format(nr=attribute_nr-2,
#                                                                        attribute=occurence.columns[attribute_nr])
#             print(message2)
#             p_value = permutation_of_individual_test(d,
#                                                      occurence,
#                                                      mdf,
#                                                      nr_donors_with_d,
#                                                      t1o,
#                                                      attribute_nr,
#                                                      m_or_m='mean')#, donor_diagnosis_list)
#             p_values_attribute_dictionary[attribute_nr] = p_value

#         p_values_diagnosis_dictionary[d] = p_values_attribute_dictionary
    
    return p_values_diagnosis_dictionary


In [ ]:
if train_plot == False:
    table_folder = "{}/{}".format(figure_folder,table_of_choice)
if train_plot == True:
    table_folder = "{}/{}_training".format(figure_folder,table_of_choice)
print(table_folder)

if not os.path.exists(table_folder):
    print('Creating output folder....')
    os.makedirs(table_folder)
    
save_permutation = "{}/p_values_{}.xlsx".format(table_folder,table_of_choice)
print(save_permutation)

In [ ]:
# check
# table_of_choice
# flattened

#### running the permutation test

In [ ]:
# Pvalues_dict = identify_diagnosis_Pvalues(ordered_diagnoses, flattened, general_mean_multi, disease_counts, flattened_t1)
# prelim = pd.DataFrame(Pvalues_dict)
# Pvalues_dataframe = prelim.T
# Pvalues_dataframe.columns = list(general_mean_multi.columns)
# display(Pvalues_dataframe)

# writer = pd.ExcelWriter(save_permutation, engine='xlsxwriter')
# Pvalues_dataframe.to_excel(writer)
# writer.save()

#### Loading stored permutation test

In [ ]:
Pvalues_dataframe = pd.read_excel(save_permutation, engine='openpyxl', index_col=[0])


In [ ]:

Pvalues_dataframe=Pvalues_dataframe.reindex(ordered_diagnoses)
display(Pvalues_dataframe)
print(Pvalues_dataframe.shape)

#### correct for multiple testing

In [ ]:
def FDR_conversion(Pvalues_dataframe):
        
    ##Function that converts a P-value dataframe to FDR dataframe 
    import statsmodels.stats.multitest as smt
    
    Pvalues_list = [] 
    for index_value in Pvalues_dataframe.index:
        Pvalues_list+= Pvalues_dataframe.loc[index_value,:].values.tolist()

    FDRvalues_list = smt.multipletests(Pvalues_list, method='fdr_bh', is_sorted= False)[1]    
    FDRvalues_array = np.array(FDRvalues_list) 
    FDRvalues_array = np.reshape(FDRvalues_array, Pvalues_dataframe.shape)
    FDR_df = pd.DataFrame(FDRvalues_array, columns= Pvalues_dataframe.columns, index= Pvalues_dataframe.index)
    
    return FDR_df
    

In [ ]:
FDR_df = FDR_conversion(Pvalues_dataframe)
FDR_Cutoff = 0.1
significance_boolean = (FDR_df < FDR_Cutoff) * 1

#### Our colleagues from the NHB wrote down their expectations for each attribute and diagnosis. we plot these as well

In [ ]:
###Add expected attributes per diagnosis - for plotting 
expected_attributes_df = pd.read_excel(expected_attributes_path, index_col=0,sheet_name='Updated version 20042022')

expected_attributes_df.fillna(0,inplace=True)
expected_attributes_df = expected_attributes_df.astype('int')
print(pd.unique(expected_attributes_df.values.ravel('K')))
expected_attributes_df = expected_attributes_df.rename(columns={'Executive_dysfunction': 'Executive_function_disorder',
                                                                'Lack_of_planning_organisation':'Lack_of_planning_organis_overv',
                                                               'Unspecified_disturbed_gait_patterns': 'Unspecified_disturbed_gait_patt'})
expected_attributes_df = expected_attributes_df.rename(columns={"Fatique": "Fatigue"})
expected_attributes_df= expected_attributes_df.rename(correct_names,axis=1)
expected_attributes_df = expected_attributes_df.reindex(ordered_diagnoses)
display(expected_attributes_df)



In [ ]:
def synonym_merger(df, how=None):
    if how == 'sum':
        df["Loss of coordination"] = df[["Ataxia", "Loss of coordination"]].sum(axis=1)
        df["Apathy / inertia"] = df[["Apathy / inertia", "Lack of initiative"]].sum(axis=1)
        df["Dementia"] = df[["Dementia", "Cognitive decline"]].sum(axis=1)
        df["Executive function disorders"] = df[["Executive function disorders", "Lack of planning / organization / overview"]].sum(axis=1)
    elif how == 'max':
        df["Loss of coordination"] = df[["Ataxia", "Loss of coordination"]].max(axis=1)
        df["Apathy / inertia"] = df[["Apathy / inertia", "Lack of initiative"]].max(axis=1)
        df["Dementia"] = df[["Dementia", "Cognitive decline"]].max(axis=1)
        df["Executive function disorders"] = df[["Executive function disorders", "Lack of planning / organization / overview"]].max(axis=1)
    df.drop(['Lack of initiative','Cognitive decline','Ataxia',"Lack of planning / organization / overview"], axis=1,inplace=True)
    return df

if train_plot == False:
    expected_attributes_df = synonym_merger(expected_attributes_df, how='max') ## turn off ## for training data
expected_attributes_df = expected_attributes_df[list(general_mean_multi.columns)]
expected_attributes_df = expected_attributes_df.fillna(0)
display(expected_attributes_df)
print(expected_attributes_df.shape)

## all dataframes are nice, now we calculate bar size based on these

In [ ]:
# VERTICAL BARPLOT
attribute_bar = pd.DataFrame(flattened[list(general_mean_multi.columns)].sum(),columns=['Attribute'])
attribute_bar['freq']=(attribute_bar['Attribute']*-2)/attribute_bar['Attribute'].max()
print('attributes min max')
print(attribute_bar['Attribute'].max())
print(attribute_bar['Attribute'].min())
print(attribute_bar['freq'].max())
print(attribute_bar['freq'].min())
display(attribute_bar)
freq =attribute_bar['freq'].tolist()

# Get positions for attribute barplots
positions = np.arange(start=.5, stop=.5*len(list(general_mean_multi.columns))*2, step=1)
print(len(positions))

### HORIZONTAL BARPLOT
disease_counts['freq']= (disease_counts['neuropathological_diagnosis']*2)/disease_counts['neuropathological_diagnosis'].max()
print('diagnoses max min')
print(disease_counts['neuropathological_diagnosis'].max())
print(disease_counts['neuropathological_diagnosis'].min())
print(disease_counts['freq'].max())
print(disease_counts['freq'].min())
display(disease_counts)
prop_freq_diag = disease_counts['freq'].tolist()
print(prop_freq_diag)

diag_pos = np.arange(start=.5, stop=.5*(len(prop_freq_diag))*2, step=1)
print(diag_pos)

In [ ]:
added_colors = []

for col in list(general_mean_multi.columns):
#     print(col)
    for g in group_dict_fancy:
#         print(g)
        if col in group_dict_fancy[g]:
#             print('grouping: {}'.format(g))
            added_colors.append(color_dict_fancy[g])
#         print('\n')
#         print(col)

print(len(added_colors))
print(added_colors)

### Plot the Heatmap

In [ ]:
plt.figure(dpi=1200)

In [ ]:
num = int(len(proportion_df.columns)/2)+2
if train_plot == False:
    num = 80
if train_plot == True:
    num = 90 ## for training data

In [ ]:
first40_colnames = proportion_df.columns[0:num]
first40_colnames = proportion_df.columns
first40_colors = added_colors[0:num]
first40_colors =  added_colors
print(first40_colnames)
print(len(first40_colnames))
print(num)
# print(proportion_df.columns)

In [ ]:
%matplotlib inline
sns.set(style="darkgrid", font_scale=1.5, rc={'axes.facecolor':'#F0E6EB', "grid.linestyle": "-","grid.color": '#b0b0b0'})

##PLOT 
w = disease_counts.shape[0]/1
w = (3+ disease_counts.shape[0]/1)-4
h = 2+num/3
w = 5.5
h= 30
fig = plt.figure(figsize=(w,h ),dpi=200)
print('figsize:',w,h)
# fig = plt.figure(figsize=(15, 20))
ax1 = plt.subplot2grid((9,9), (0, 0), colspan=8,rowspan=9)

# #Dots for expected attributes per disease
# ax1 = scattermap(expected_attributes_df[first40_colnames].T,
#                 marker='o',
#                 marker_size=proportion_df[first40_colnames].T*expected_attributes_df[first40_colnames].T * 1.3,
#                 cmap="Oranges",
#                 cbar=False,ax=ax1)

##Dots for proportions and averages 
ax1 = scattermap(general_mean_multi[first40_colnames].T,
                cmap="YlGnBu",
                marker_size=proportion_df[first40_colnames].T,
                ax=ax1,
                vmax=5,
                 linecolor = 'black',
                 linewidths = 0.2,
                 
                cbar_kws={"shrink": .5})#cbar_kws = {"orientation": "horizontal", "pad":0.02}
#                 cbar_kws = dict(use_gridspec=True,location="right"))#,
#                 cbar=False)

#Significance 
ax1 = scattermap(significance_boolean[first40_colnames].T,
                marker='*',
                marker_size=significance_boolean[first40_colnames].T * 100,
                cbar=False,
                ax=ax1,
                 linecolor = 'black',
                 linewidths = 0.2,
                cmap="Wistia")

# x axis on top
ax1.xaxis.tick_top() 
ax1.xaxis.set_label_position('top')
# plt.xticks(rotation=75) (old, no subplot)
ax1.tick_params('x', labelrotation=90)

# Add frequencies of attributes as barplot to y-axis
ax1.barh(list(positions)[:num], freq[0:num], 0.6, alpha=1, color=first40_colors,edgecolor = "none")
# plt.axvline(x=0, color='k') (old, no subplot)
ax1.axvline(x=0, color='k')
ax1.axhline(0, color='k')

#plt.xlim(attribute_bar['freq'].min()-0.1,disease_counts.shape[0])(old, no subplot)
ax1.set_xlim([attribute_bar['freq'].min()-0.1,disease_counts.shape[0]])

# Add frequencies of diagnosis as barplot to x-axis
ax1.bar(diag_pos, prop_freq_diag, 0.6,color='#41b6c4',bottom=num,edgecolor = "none")

# plt.ylim(0,80+disease_counts['freq'].max()+0.1)(old, no subplot)
ax1.set_ylim([0,num+disease_counts['freq'].max()+0.1])

plt.title('{}: {} to {} diagnoses, {} to {} attributes, for {} donors'.format(table_of_choice,
                                                                             disease_counts['neuropathological_diagnosis'].min(),
                                                                             disease_counts['neuropathological_diagnosis'].max(),
                                                                             attribute_bar['Attribute'].min(),
                                                                             attribute_bar['Attribute'].max(),
                                                                             selected_donorcountpredict))

## same size legend
ax2 = plt.subplot2grid((9, 9), (3, 8),colspan=1,rowspan=1)
x = [2,3,4,5]
y = [2,3,4,5]
a2 = [75,150,225,300]

sc = ax2.scatter(x, y, s=a2, alpha=0.5,c='white')
L = ax2.legend(*sc.legend_elements("sizes"),loc='center left', bbox_to_anchor=(1, 0.7),frameon=False)
L.get_texts()[0].set_text('25%')
L.get_texts()[1].set_text('50%')
L.get_texts()[2].set_text('75%')
L.get_texts()[3].set_text('100%')

ax2.axis('off')

ax3 = plt.subplot2grid((9, 9), (2, 8),colspan=1,rowspan=1)
x2 = [2]
x3 = [2]
y2 = [2]
y3 = [2]
a3 = [300]
t = ['significance','expected']
ax3.scatter(x2, y2, s=a3, alpha=1,c='#E59400', marker='*')
ax3.scatter(x3, y3, s=a3, alpha=1,facecolors='none',edgecolors='#E59400')
ax3.legend(t, loc='center left',bbox_to_anchor=(1, 0.3),frameon=False)
ax3.set_ylim([0,1])
ax3.set_xlim([0,1])
ax3.axis('off')

print('based on table {}'.format(table_of_choice))
print(disease_counts.shape[0])
print('{} to {} diagnosis, {} to {} attributes'.format(disease_counts['neuropathological_diagnosis'].min(),
                                                         disease_counts['neuropathological_diagnosis'].max(),
                                                         attribute_bar['Attribute'].min(),
                                                         attribute_bar['Attribute'].max() ))
plt.tight_layout()
plt.savefig("{}/dotplot_{}.png".format(table_folder,table_of_choice), bbox_inches="tight")
plt.savefig("{}/dotplot_{}.pdf".format(table_folder,table_of_choice), bbox_inches="tight")

plt.show()
plt.close()

In [ ]:
def chi_square_testing_of_dataframes(expected_attributes_df, significance_boolean_df):
    ##Function to calculate the Chi-square statistic to deteremine the overlap between the significance boolean dataframe and expected dataframe
    import scipy
    ###Chi-square testing
    boolean_matrix = (expected_attributes_df == 1) & (significance_boolean_df ==1)
    category00 = np.sum(np.sum((boolean_matrix * 1)))
    boolean_matrix = (expected_attributes_df == 0) & (significance_boolean_df ==1)
    category01 = np.sum(np.sum((boolean_matrix * 1)))
    boolean_matrix = (expected_attributes_df == 1) & (significance_boolean_df ==0)
    category10 = np.sum(np.sum((boolean_matrix * 1)))
    boolean_matrix = (expected_attributes_df == 0) & (significance_boolean_df ==0)
    category11 = np.sum(np.sum((boolean_matrix * 1)))
    observations = np.array([[category00,category01], [category10, category11]])
    chi2, p, dof, expected = scipy.stats.chi2_contingency(observations)
    print(f"chi2 statistic:   {chi2:.5g}")
    print(f"p-value:      {p:.5g}")
    print(f"degrees of freedom: {dof}")
    print()
    print("expected frequencies:")
    print(expected)
    print()
    print("observed frequencies:")
    print(observations)
    
chi_square_testing_of_dataframes(expected_attributes_df, significance_boolean)